In [1]:
!pip install datasets
!pip install transformers
!pip install torch
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [2]:
from datasets import load_dataset
dataset = load_dataset("gjyotk/Menstrual-Health-Awareness-Dataset")
print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/530 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/45 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction (string)', 'output (string)'],
        num_rows: 530
    })
    test: Dataset({
        features: ['instruction (string)', 'output (string)'],
        num_rows: 45
    })
})


In [34]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")



In [35]:
def tokenize_data(example):
    input_text = example["instruction (string)"]
    output_text = example["output (string)"]

    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=530)
    outputs = tokenizer(output_text, truncation=True, padding="max_length", max_length=45)

    inputs["labels"] = outputs["input_ids"]
    return inputs


In [80]:

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_data(example):
    input_text = example["instruction (string)"]
    output_text = example["output (string)"]


    if input_text is None or len(input_text) == 0:
        input_text = " "
    if output_text is None or len(output_text) == 0:
        output_text = " "


    inputs = tokenizer(input_text, truncation=True, padding='max_length', max_length=512)
    outputs = tokenizer(output_text, truncation=True, padding='max_length', max_length=512)

    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"],
            "labels": outputs["input_ids"]}



tokenized_train_dataset = train_dataset.map(tokenize_data)
tokenized_validation_dataset = validation_dataset.map(tokenize_data)


print(tokenized_train_dataset[0])
print(tokenized_validation_dataset[0])


Map:   0%|          | 0/530 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

{'instruction (string)': 'What is a normal menstrual cycle length?', 'output (string)': 'A normal menstrual cycle typically ranges from 21 to 35 days, with the average cycle lasting around 28 days.', 'input_ids': [2061, 318, 257, 3487, 37230, 6772, 4129, 30, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257, 50257,

In [53]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))


Embedding(50258, 768)

In [54]:
import torch
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments


class CustomDataCollator(DataCollatorForSeq2Seq):
    def collate_batch(self, features):
        batch = super().collate_batch(features)

        # Move tensors to the appropriate device
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch[k] = v.to(self.device)
            elif isinstance(v, list):
                batch[k] = [None if item is None else item.to(self.device) for item in v]

        return batch


training_args = TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    output_dir="./output",
    num_train_epochs=3,
    overwrite_output_dir=True,
)


data_collator = CustomDataCollator(tokenizer)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    data_collator=data_collator,
)


trainer.train()


eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.424638
2,No log,0.428227
3,No log,0.416954


Evaluation results: {'eval_loss': 0.4169544279575348, 'eval_runtime': 2.2102, 'eval_samples_per_second': 20.361, 'eval_steps_per_second': 5.429, 'epoch': 3.0}


In [69]:
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")


('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.json',
 'saved_model/merges.txt',
 'saved_model/added_tokens.json')

In [79]:
import torch

def generate_output(input_instruction, max_length=45, temperature=0.7, top_k=0, top_p=0.9):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)


    input_ids = tokenizer(input_instruction, return_tensors="pt").input_ids.to(device)


    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p
    )
    output_ids = output_ids.cpu()

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

input_instruction = "What is a normal menstrual cycle length?"
generated_output = generate_output(input_instruction, max_length=32)
print("Generated output:", generated_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated output: What is a normal menstrual cycle length?

A normal menstrual cycle length is the length of time between the first menstrual cycle and the last menstrual cycle.

